<img src="https://unskript.com/assets/favicon.png" alt="unSkript.com" width="100" height="100"/> 
<h1> unSkript Runbooks </h1>
<div class="alert alert-block alert-success">
    <b> This runbook demonstrates How to restart a unHealthy service target group using unSkript legos.</b>
</div>

<br>

<center><h2>Restart unHealthy Service Target Group</h2></center>

# Steps Overview
1) Get the list of Target group matching the tag and their corresponding tag value (which is the restart command). Store this in targetGroupARNs, which is a list of targetGroupARN and corresponding restart command.
2) Get the list of unhealthy instances behind each target group. Store this in an array named unHealthyInstanceIDs. This contains the TargetGroupARN, InstanceID, RestartCommand.
3) For each instance in unHealthyInstanceIDs, get their public/private IP. Store this in IPsAndRestartCommandList.
4) Run remote ssh command for each entry in IPsAndRestartCommandList.
5) Create per target group ARN Slack message and store it in messages.
6) Iterate over messages and send one by one on slack.

Here we will use unSkript Filter AWS Target groups by tag name Lego. This lego takes tag_key and region as input. This inputs is used to Filter AWS Target groups which have the provided tag attached to it. It also returns the value of that tag for each target group.

In [ ]:
##
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field
from typing import List
from unskript.connectors.aws import aws_get_paginator

from beartype import beartype
@beartype
def aws_filter_target_groups_by_tags(handle, tag_key: str, region: str) -> List:
    elbv2Client = handle.client('elbv2', region_name=region)
    tbs = aws_get_paginator(elbv2Client, "describe_target_groups", "TargetGroups")
    tbArnsList = []
    output = []
    count = 0
    tbsLength = len(tbs)
    for index, tb in enumerate(tbs):
        # Need to call describe_tags to get the tags associated with these TGs, however that call can only take 20 TGs.
        tbArnsList.append(tb.get('TargetGroupArn'))
        count = count + 1
        if count == 20 or index == tbsLength - 1:
            tagDescriptions = elbv2Client.describe_tags(ResourceArns=tbArnsList).get('TagDescriptions')
            # Check if the tag name exists in any of the TGs.
            for tagDescription in tagDescriptions:
                for tag in tagDescription.get('Tags'):
                    if tag.get('Key') == tag_key:
                        output.append({"ResourceARN": tagDescription.get('ResourceArn'), "TagValue": tag.get('Value')})
                        break
            count = 0
            tbArnsList = []
    return output



task = Task(Workflow())
task.configure(inputParamsJson='''{
    "region": "Region",
    "tag_key": "RestartCommandTagName"
    }''')
task.configure(outputName="targetGroupARNs")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.output = task.execute(aws_filter_target_groups_by_tags, hdl=hdl, args=args)
    if task.output_name != None:
        globals().update({task.output_name: task.output[0]})

if hasattr(task, 'output'):
    if isinstance(task.output, (list, tuple)):
        for item in task.output:
            print(f'item: {item}')
    elif isinstance(task.output, dict):
        for item in task.output.items():
            print(f'item: {item}')
    else:
        print(f'Output for {task.name}')
        print(task.output)
    w.tasks[task.name]= task.output

Here we will use unSkript AWS List Unhealthy Instances in a ELBV2 Target Group Lego. This lego takes arn as input. This inputs is used to List AWS Unhealthy Instance in a ELBv2 Target Group.

In [ ]:
##
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field
from unskript.legos.utils import parseARN


from beartype import beartype
@beartype
def aws_target_group_list_unhealthy_instances(handle, arn: str):
    # Get the region for the target group.
    parsedArn = parseARN(arn)
    elbv2Client = handle.client('elbv2', region_name=parsedArn['region'])
    try:
        targetHealthResponse = elbv2Client.describe_target_health(
            TargetGroupArn=arn
        )
    except Exception as e:
        print(f'Hit exception getting the instance list: {str(e)}')
        raise e

    instancesInfo = []
    for ins in targetHealthResponse["TargetHealthDescriptions"]:
        if ins['TargetHealth']['State'] in ['unhealthy']:
            instancesInfo.append(ins)

    return instancesInfo


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "arn": "iter.get(\\"ResourceARN\\")"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "targetGroupARNs",
    "iter_parameter": ["arn"]
    }''')
task.configure(outputName="unHealthyInstanceOutput")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.output = task.execute(aws_target_group_list_unhealthy_instances, hdl=hdl, args=args)
    if task.output_name != None:
        globals().update({task.output_name: task.output[0]})

if hasattr(task, 'output'):
    if isinstance(task.output, (list, tuple)):
        for item in task.output:
            print(f'item: {item}')
    elif isinstance(task.output, dict):
        for item in task.output.items():
            print(f'item: {item}')
    else:
        print(f'Output for {task.name}')
        print(task.output)
    w.tasks[task.name]= task.output

In [29]:
# Check if there are any unhealthy targets
unHealthyInstanceIDs = []
for k, v in unHealthyInstanceOutput.items():
    if len(v) == 0:
        print(f'No unhealthy instances in {targetGroupARNs[k].get("ResourceARN")}')
        continue
    for i in v:
        target = i.get('Target')
        if target is None:
            continue
        instanceID = target.get('Id')
        if instanceID is not None:
            unHealthyInstanceIDs.append({"TargetGroupARN":targetGroupARNs[k].get("ResourceARN"), "InstanceID": instanceID, "RestartCommand": targetGroupARNs[k].get('TagValue')})


Here we will use unSkript Get AWS Instance Details Lego. This lego takes InstanceID and region as input. This inputs is used to Get Details about an AWS EC2 Instance.

In [ ]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
from pydantic import BaseModel, Field


from beartype import beartype
@beartype
def aws_get_instance_details(
    handle,
    instance_id: str,
    region: str,
):

    ec2client = handle.client('ec2', region_name=region)
    instances = []
    response = ec2client.describe_instances(
        Filters=[{"Name": "instance-id", "Values": [instance_id]}])
    for reservation in response["Reservations"]:
        for instance in reservation["Instances"]:
            instances.append(instance)

    return instances[0]


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "instance_id": "\\"iter.get(\\\\\\"InstanceID\\\\\\")\\"",
    "region": "\\"us-west-2\\""
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "unHealthyInstanceIDs",
    "iter_parameter": ["instance_id"]
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "len(unHealthyInstanceIDs) > 0",
    "condition_result": true
}''')
task.configure(outputName="InstanceDetailOutput")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.output = task.execute(aws_get_instance_details, hdl=hdl, args=args)
    if task.output_name != None:
        globals().update({task.output_name: task.output[0]})

if hasattr(task, 'output'):
    if isinstance(task.output, (list, tuple)):
        for item in task.output:
            print(f'item: {item}')
    elif isinstance(task.output, dict):
        for item in task.output.items():
            print(f'item: {item}')
    else:
        print(f'Output for {task.name}')
        print(task.output)
    w.tasks[task.name]= task.output

In [31]:
IPsAndRestartCommandList = []
if len(unHealthyInstanceIDs) > 0:
    for key, value in InstanceDetailOutput.items():
        ips = []
        if UsePublicIP:
            ips.append(value.get('PublicIpAddress'))
        else:
            ips.append(value.get('PrivateIpAddress'))
        IPsAndRestartCommandList.append({"TargetGroupARN": unHealthyInstanceIDs[key].get("TargetGroupARN"), "InstanceID": unHealthyInstanceIDs[key].get("InstanceID"), "IP": ips, "RestartCommand": unHealthyInstanceIDs[key].get('RestartCommand')})


Here we will use unSkript SSH Execute Remote Command Lego. This lego takes hosts, command and sudo as input. This input is used to SSH Execute restart Commands.

In [ ]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
import json
import tempfile
import os
from pydantic import BaseModel, Field
from pssh.clients import ParallelSSHClient
from typing import List, Optional
from unskript.connectors import ssh

from unskript.legos.cellparams import CellParams
from unskript import connectors


from beartype import beartype
@beartype
def ssh_execute_remote_command(sshClient, hosts: List[str], command: str, sudo: bool = False):

    client = sshClient(hosts)
    runCommandOutput = client.run_command(command=command, sudo=sudo)
    client.join()
    res = {}

    for host_output in runCommandOutput:
        hostname = host_output.host
        output = []
        for line in host_output.stdout:
            output.append(line)
        res[hostname] = output

        o = "\n".join(output)
        print(f"Output from host {hostname}\n{o}\n")

    return res


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "sudo": "True",
    "command": "iter.get(\\"RestartCommand\\")",
    "hosts": "iter.get(\\"IP\\")"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "IPsAndRestartCommandList",
    "iter_parameter": ["command","hosts"]
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "len(unHealthyInstanceIDs) > 0",
    "condition_result": true
}''')

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.output = task.execute(ssh_execute_remote_command, hdl=hdl, args=args)
    if task.output_name != None:
        globals().update({task.output_name: task.output[0]})

if hasattr(task, 'output'):
    if isinstance(task.output, (list, tuple)):
        for item in task.output:
            print(f'item: {item}')
    elif isinstance(task.output, dict):
        for item in task.output.items():
            print(f'item: {item}')
    else:
        print(f'Output for {task.name}')
        print(task.output)
    w.tasks[task.name]= task.output

Here the Action creates a per target group ARN Slack message and stores it in `messages`.

In [33]:
messages = []
tgInstanceMap = {}
for i in IPsAndRestartCommandList:
    if tgInstanceMap.get(i.get('TargetGroupARN')) is None:
        tgInstanceMap[i.get('TargetGroupARN')] = [i.get('InstanceID')]
    else:
        currentList = tgInstanceMap[i.get('TargetGroupARN')]
        currentList.append(i.get('InstanceID'))
        tgInstanceMap[i.get('TargetGroupARN')] = currentList

for key, value in tgInstanceMap.items():
    printInstanceIDs = "\n".join(value)
    messages.append(f'Services in the following instances in target group {key} were restarted: \n{printInstanceIDs}')

Here we will use unSkript Post Slack Message Lego. This lego takes channel: str and message: str as input. This inputs is used to post the message to the slack channel.

In [ ]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##

import pprint

from pydantic import BaseModel, Field
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError

pp = pprint.PrettyPrinter(indent=2)


from beartype import beartype
def legoPrinter(func):
    def Printer(*args, **kwargs):
        output = func(*args, **kwargs)
        if output:
            channel = kwargs["channel"]
            pp.pprint(print(f"Message sent to Slack channel {channel}"))
        return output
    return Printer


@legoPrinter
@beartype
def slack_post_message(
        handle: WebClient,
        channel: str,
        message: str) -> bool:

    try:
        response = handle.chat_postMessage(
            channel=channel,
            text=message)
        return True
    except SlackApiError as e:
        print("\n\n")
        pp.pprint(
            f"Failed sending message to slack channel {channel}, Error: {e.response['error']}")
        return False
    except Exception as e:
        print("\n\n")
        pp.pprint(
            f"Failed sending message to slack channel {channel}, Error: {e.__str__()}")
        return False


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "channel": "\\"dummy\\"",
    "message": "iter_item"
    }''')

task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "len(messages) != 0",
    "condition_result": true
}''')

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.output = task.execute(slack_post_message, hdl=hdl, args=args)
    if task.output_name != None:
        globals().update({task.output_name: task.output[0]})

### Conclusion
In this Runbook, we demonstrated the use of unSkript's AWS legos to Restart unhealthy services in a Target Group and restarts unhealthy services in a target group. The restart command is provided via a tag attached to the instance. To view the full platform capabilities of unSkript please visit https://us.app.unskript.io